<a href="https://colab.research.google.com/github/Trading-com-Dados/desenvolvimento_interno/blob/main/JPJ/20220523_Code_VectorBT_Trading_com_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://tradingcomdados.com/images/logotipo/logotipo-trading-com-dados.svg" width="300" align="left"/>

# **Biblioteca VectorBT: análises quantitativas de forma simples e rápida**

# **1. Bibliotecas utilizadas**

In [75]:
!pip install vectorbt
!pip install yfinance
!pip install investpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [183]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pytz

import yfinance as yf
import investpy as inv
import vectorbt as vbt

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

# **2. Primeiros passos com a *VectorBT***

## 2.1. Baixar os dados de cotações - YFINANCE

A vectorbt foi criada para interagir muito bem com Numpy e Numba, além de integrar a Plotly e as ferramentas do Jupyter.<p>
Ao mesmo tempo, trabalha muito bem com a Pandas

In [77]:
# Baixando os dados do YF (timezone UTC)

btc_ohlc = vbt.YFData.download('BTC-USD', period = 'max',interval='1d').get(['Open','High','Low','Close','Volume'])
btc_ohlc

,Open,High,Low,Close,Volume
Date,,,,,
2014-09-17 00:00:00+00:00,465.864014,468.174011,452.421997,457.334015,21056800
2014-09-18 00:00:00+00:00,456.859985,456.859985,413.104004,424.440002,34483200
2014-09-19 00:00:00+00:00,424.102997,427.834991,384.532013,394.795990,37919700
2014-09-20 00:00:00+00:00,394.673004,423.295990,389.882996,408.903992,36863600
2014-09-21 00:00:00+00:00,408.084991,412.425995,393.181000,398.821014,26580100
...,...,...,...,...,...
2022-05-20 00:00:00+00:00,30311.119141,30664.976562,28793.605469,29200.740234,30749382605
2022-05-21 00:00:00+00:00,29199.859375,29588.869141,29027.394531,29432.226562,17274840442
2022-05-22 00:00:00+00:00,29432.472656,30425.861328,29275.183594,30323.722656,21631532270


In [78]:
# Neste primeiro exemplo vamos focar no preço de fechamento -> Close

btc_price = vbt.YFData.download('BTC-USD', period = 'max',interval='1d').get('Close')
btc_price

Date
2014-09-17 00:00:00+00:00      457.334015
2014-09-18 00:00:00+00:00      424.440002
2014-09-19 00:00:00+00:00      394.795990
2014-09-20 00:00:00+00:00      408.903992
2014-09-21 00:00:00+00:00      398.821014
                                 ...     
2022-05-20 00:00:00+00:00    29200.740234
2022-05-21 00:00:00+00:00    29432.226562
2022-05-22 00:00:00+00:00    30323.722656
2022-05-23 00:00:00+00:00    29098.910156
2022-05-24 00:00:00+00:00    28941.642578
Freq: D, Name: Close, Length: 2807, dtype: float64

## 2.2. Baixar os dados de cotações - INVESTPY

In [79]:
# Estrutura padrão pandas p/ obter dados da Investing (investpy) e por fim método 'get' da vectorbt

start = '01/01/2015'
end = '31/12/2021'
BVSP_search_result = inv.search_quotes(text='Bovespa (BVSP)', products=['indices'], n_results=1)
print(BVSP_search_result)

BVSP_data = BVSP_search_result.retrieve_historical_data(from_date=start, to_date=end).get('Close')
BVSP_data

{"id_": 17920, "name": "Bovespa", "symbol": "BVSP", "country": "brazil", "tag": "/indices/bovespa", "pair_type": "indices", "exchange": "BM&FBovespa"}


Date
2015-01-02     48512.0
2015-01-05     47517.0
2015-01-06     48001.0
2015-01-07     49463.0
2015-01-08     49943.0
                ...   
2021-12-23    104891.0
2021-12-27    105554.0
2021-12-28    104864.0
2021-12-29    104107.0
2021-12-30    104822.0
Name: Close, Length: 1730, dtype: float64

## 2.3. Análise simplificada de HOLDING

Aqui um exemplo claro de porque a vectorbt é considerada uma das mais simples e fáceis para análises quant

In [88]:
# Métodos Portfolio.from_holding e total_profit
# Qt teríamos no final de 2021 se tivéssemos comprando $100 em BTC-USD no inicio de 2015

start = '2015-01-01'
end = '2021-12-31'
btc_price = vbt.YFData.download('BTC-USD', start=start, end=end,interval='1d').get('Close')
btc_price

Date
2015-01-01 00:00:00+00:00      314.248993
2015-01-02 00:00:00+00:00      315.032013
2015-01-03 00:00:00+00:00      281.082001
2015-01-04 00:00:00+00:00      264.195007
2015-01-05 00:00:00+00:00      274.473999
                                 ...     
2021-12-27 00:00:00+00:00    50640.417969
2021-12-28 00:00:00+00:00    47588.855469
2021-12-29 00:00:00+00:00    46444.710938
2021-12-30 00:00:00+00:00    47178.125000
2021-12-31 00:00:00+00:00    46306.445312
Freq: D, Name: Close, Length: 2557, dtype: float64

In [89]:
pf = vbt.Portfolio.from_holding(btc_price, init_cash=100)
pf.total_profit()

14635.590679935762

## 2.4. Backtesting ESTRATÉGIA Cruzamento de Médias Móveis

Comprar se MM rápida cruzar para cima a MM lenta, e vender quando oposto acontecer:

In [90]:
# Criando as médias móveis (módulo vbt indicators)

fast_ma = vbt.MA.run(btc_price, 10)
slow_ma = vbt.MA.run(btc_price, 50)
slow_ma

In [91]:
# Checando as entradas e saídas, quando houver o cruzamento (output booleano)

entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)
entries

Date
2015-01-01 00:00:00+00:00    False
2015-01-02 00:00:00+00:00    False
2015-01-03 00:00:00+00:00    False
2015-01-04 00:00:00+00:00    False
2015-01-05 00:00:00+00:00    False
                             ...  
2021-12-27 00:00:00+00:00    False
2021-12-28 00:00:00+00:00    False
2021-12-29 00:00:00+00:00    False
2021-12-30 00:00:00+00:00    False
2021-12-31 00:00:00+00:00    False
Freq: D, Length: 2557, dtype: bool

In [92]:
# Vejamos alguns pontos de cruzamento MM10 > MM50

entries[entries==True].head()

Date
2015-02-21 00:00:00+00:00    True
2015-05-11 00:00:00+00:00    True
2015-06-17 00:00:00+00:00    True
2015-10-01 00:00:00+00:00    True
2016-02-20 00:00:00+00:00    True
dtype: bool

In [93]:
# Cálculo Médias Móveis (10 e 50 períodos) com pandas
# Estamos usando os dados baixados no início desse código

btc_ohlc['MMA10'] = btc_ohlc['Close'].rolling(window=10).mean()
btc_ohlc['MMA50'] = btc_ohlc['Close'].rolling(window=50).mean()
btc_ohlc.head()

,Open,High,Low,Close,Volume,MMA10,MMA50
Date,,,,,,,
2014-09-17 00:00:00+00:00,465.864014,468.174011,452.421997,457.334015,21056800,NaN,NaN
2014-09-18 00:00:00+00:00,456.859985,456.859985,413.104004,424.440002,34483200,NaN,NaN
2014-09-19 00:00:00+00:00,424.102997,427.834991,384.532013,394.795990,37919700,NaN,NaN
2014-09-20 00:00:00+00:00,394.673004,423.295990,389.882996,408.903992,36863600,NaN,NaN
2014-09-21 00:00:00+00:00,408.084991,412.425995,393.181000,398.821014,26580100,NaN,NaN


In [94]:
# Checagem gráfica

data_figure = btc_ohlc[(btc_ohlc.index>='2015-02-15') & (btc_ohlc.index<='2015-02-25')]

fig = go.Figure(go.Candlestick(name='BTC-USD', x=data_figure.index, open=data_figure['Open'], high = data_figure['High'],
                               low=data_figure['Low'], close=data_figure['Close'],showlegend=False))

fig.add_trace(go.Scatter(name='MMA10', x=data_figure.index, y=data_figure['MMA10'],marker_color='red'))
fig.add_trace(go.Scatter(name='MMA50', x=data_figure.index, y=data_figure['MMA50'],marker_color='blue'))

fig.update_layout(title='', xaxis_title="<b>Data", yaxis_title="<b>Bitcoin (USD)", legend_title="",
                  font=dict(family="Arial, Arial, Arial",size=15,color="black"))

fig.update_layout(xaxis_rangeslider_visible=False, title_text='Cruzamento MM10 e MM50',width=500,height=400,
                  template = 'simple_white',margin=dict(l=50, r=20, t=50, b=20),paper_bgcolor="#f7f8fa")
fig.show()

In [95]:
# E aqui checamos o resultado do backtesting, considerando o cruzamento das duas médias móveis, ao longo do período avaliado e investindo $100

pf = vbt.Portfolio.from_signals(btc_price, entries, exits, init_cash=100)
pf.total_profit()

21219.59300155006

## 2.5. Aumentar a complexidade do backtesting Cruzamento de Médias Móveis

Podemos adicionar outras estratégias de forma muito simples<p>

Nesse caso, passaremos uma janela de períodos para as MM ao invés de um único valor

In [113]:
# Exemplo com múltiplas médias rápidas a serem avaliadas

fast_ma = vbt.MA.run(btc_price, [9, 10, 20, 30], short_name='fast')
slow_ma = vbt.MA.run(btc_price, [50, 50, 50, 50], short_name='slow')
fast_ma

In [114]:
entries = fast_ma.ma_crossed_above(slow_ma)
entries

fast_window,9,10,20,30
slow_window,50,50,50,50
Date,,,,
2015-01-01 00:00:00+00:00,False,False,False,False
2015-01-02 00:00:00+00:00,False,False,False,False
2015-01-03 00:00:00+00:00,False,False,False,False
2015-01-04 00:00:00+00:00,False,False,False,False
2015-01-05 00:00:00+00:00,False,False,False,False
...,...,...,...,...
2021-12-27 00:00:00+00:00,False,False,False,False
2021-12-28 00:00:00+00:00,False,False,False,False


In [115]:
exits = fast_ma.ma_crossed_below(slow_ma)
exits

fast_window,9,10,20,30
slow_window,50,50,50,50
Date,,,,
2015-01-01 00:00:00+00:00,False,False,False,False
2015-01-02 00:00:00+00:00,False,False,False,False
2015-01-03 00:00:00+00:00,False,False,False,False
2015-01-04 00:00:00+00:00,False,False,False,False
2015-01-05 00:00:00+00:00,False,False,False,False
...,...,...,...,...
2021-12-27 00:00:00+00:00,False,False,False,False
2021-12-28 00:00:00+00:00,False,False,False,False


In [116]:
# Método from_signals apresenta o resultado em porcentagem %

pf = vbt.Portfolio.from_signals(btc_price, entries, exits, init_cash=100)
pf.total_return()

fast_window  slow_window
9            50             193.329884
10           50             212.195930
20           50             150.120170
30           50             145.154655
Name: total_return, dtype: float64

E agora incluímos outro ativo (Ethereum) para avaliar a estratégia em múltiplos ativos e múltiplas MMs 

In [168]:
# Buscando dados Ethereum
# Importante que todos os ativos possuam o mesmo range de datas para comparação

start = '2018-01-01'
end = '2021-12-31'

btc_price = vbt.YFData.download('BTC-USD', start=start, end=end,interval='1d').get('Close')
eth_price = vbt.YFData.download('ETH-USD', start=start, end=end,interval='1d').get('Close')
comb_price = btc_price.vbt.concat(eth_price, keys=pd.Index(['BTC', 'ETH'], name='symbol'))
comb_price.vbt.drop_levels(-1, inplace=True)
comb_price

symbol,BTC,ETH
Date,,
2018-01-01 00:00:00+00:00,13657.200195,772.640991
2018-01-02 00:00:00+00:00,14982.099609,884.443970
2018-01-03 00:00:00+00:00,15201.000000,962.719971
2018-01-04 00:00:00+00:00,15599.200195,980.921997
2018-01-05 00:00:00+00:00,17429.500000,997.719971
...,...,...
2021-12-27 00:00:00+00:00,50640.417969,4037.547607
2021-12-28 00:00:00+00:00,47588.855469,3800.893066
2021-12-29 00:00:00+00:00,46444.710938,3628.531738


In [173]:
fast_ma = vbt.MA.run(comb_price, [9,10,20,30], short_name='fast')
slow_ma = vbt.MA.run(comb_price, [50,50,50,50], short_name='slow')

In [174]:
entries = fast_ma.ma_crossed_above(slow_ma)
entries

fast_window                   9             10            20            30  \
slow_window                   50            50            50            50   
symbol                       BTC    ETH    BTC    ETH    BTC    ETH    BTC   
Date                                                                         
2018-01-01 00:00:00+00:00  False  False  False  False  False  False  False   
2018-01-02 00:00:00+00:00  False  False  False  False  False  False  False   
2018-01-03 00:00:00+00:00  False  False  False  False  False  False  False   
2018-01-04 00:00:00+00:00  False  False  False  False  False  False  False   
2018-01-05 00:00:00+00:00  False  False  False  False  False  False  False   
...                          ...    ...    ...    ...    ...    ...    ...   
2021-12-27 00:00:00+00:00  False  False  False  False  False  False  False   
2021-12-28 00:00:00+00:00  False  False  False  False  False  False  False   
2021-12-29 00:00:00+00:00  False  False  False  False  False  False  False   
2021-12-30 00:00:00+00:00  False  False  False  False  False  False  False   
2021-12-31 00:00:00+00:00  False  False  False  False  False  False  False   

fast_window                       
slow_window                       
symbol                       ETH  
Date                              
2018-01-01 00:00:00+00:00  False  
2018-01-02 00:00:00+00:00  False  
2018-01-03 00:00:00+00:00  False  
2018-01-04 00:00:00+00:00  False  
2018-01-05 00:00:00+00:00  False  
...                          ...  
2021-12-27 00:00:00+00:00  False  
2021-12-28 00:00:00+00:00  False  
2021-12-29 00:00:00+00:00  False  
2021-12-30 00:00:00+00:00  False  
2021-12-31 00:00:00+00:00  False  

[1461 rows x 8 columns]

In [175]:
exits = fast_ma.ma_crossed_below(slow_ma)
exits

fast_window                   9             10            20            30  \
slow_window                   50            50            50            50   
symbol                       BTC    ETH    BTC    ETH    BTC    ETH    BTC   
Date                                                                         
2018-01-01 00:00:00+00:00  False  False  False  False  False  False  False   
2018-01-02 00:00:00+00:00  False  False  False  False  False  False  False   
2018-01-03 00:00:00+00:00  False  False  False  False  False  False  False   
2018-01-04 00:00:00+00:00  False  False  False  False  False  False  False   
2018-01-05 00:00:00+00:00  False  False  False  False  False  False  False   
...                          ...    ...    ...    ...    ...    ...    ...   
2021-12-27 00:00:00+00:00  False  False  False  False  False  False  False   
2021-12-28 00:00:00+00:00  False  False  False  False  False  False  False   
2021-12-29 00:00:00+00:00  False  False  False  False  False  False  False   
2021-12-30 00:00:00+00:00  False  False  False  False  False  False  False   
2021-12-31 00:00:00+00:00  False  False  False  False  False  False  False   

fast_window                       
slow_window                       
symbol                       ETH  
Date                              
2018-01-01 00:00:00+00:00  False  
2018-01-02 00:00:00+00:00  False  
2018-01-03 00:00:00+00:00  False  
2018-01-04 00:00:00+00:00  False  
2018-01-05 00:00:00+00:00  False  
...                          ...  
2021-12-27 00:00:00+00:00  False  
2021-12-28 00:00:00+00:00  False  
2021-12-29 00:00:00+00:00  False  
2021-12-30 00:00:00+00:00  False  
2021-12-31 00:00:00+00:00  False  

[1461 rows x 8 columns]

In [176]:
pf = vbt.Portfolio.from_signals(comb_price, entries, exits)
pf.total_return()

fast_window  slow_window  symbol
9            50           BTC        4.721492
                          ETH       11.196767
10           50           BTC        5.787394
                          ETH       10.115474
20           50           BTC        4.976394
                          ETH       10.429002
30           50           BTC        7.996978
                          ETH        5.759516
Name: total_return, dtype: float64

In [180]:
mean_return = pf.total_return().groupby('symbol').mean()
mean_return

symbol
BTC    5.870565
ETH    9.375190
Name: total_return, dtype: float64

In [182]:
mean_return.vbt.barplot(xaxis_title='Symbol', yaxis_title='Mean total return')

FigureWidget({
    'data': [{'name': 'total_return',
              'showlegend': True,
              'type': '…

Not only strategies and instruments can act as separate features, but also time. If we want to find out when our strategy performs best, it's reasonable to backtest over multiple time periods. vectorbt allows us to split one time period into many, given they have the same length and frequency, and represent them as distinct columns. For example, let's split the whole time period into two equal time periods and backest them at once.

In [ ]:
# Multiple strategy instances, instruments, and time periods
mult_comb_price, _ = comb_price.vbt.range_split(n=2)
mult_comb_price

split_idx             0                          1             
symbol              BTC          ETH           BTC          ETH
0          29374.152344   730.367554  34668.546875  2226.114258
1          32127.267578   774.534973  35287.781250  2321.724121
2          32782.023438   975.507690  33746.003906  2198.582520
3          31971.914062  1040.233032  34235.195312  2324.679443
4          33992.429688  1100.006104  33855.328125  2315.161865
..                  ...          ...           ...          ...
177        34649.644531  1978.894653  50640.417969  4037.547607
178        34434.335938  2079.657471  47588.855469  3800.893066
179        35867.777344  2160.768311  46444.710938  3628.531738
180        35040.835938  2274.547607  47178.125000  3713.852051
181        33572.117188  2113.605469  46306.445312  3682.632812

[182 rows x 4 columns]

In [ ]:
fast_ma = vbt.MA.run(mult_comb_price, [10, 20], short_name='fast')
slow_ma = vbt.MA.run(mult_comb_price, [30, 30], short_name='slow')

entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)

pf = vbt.Portfolio.from_signals(mult_comb_price, entries, exits, freq='1D')
pf.total_return()

Notice how index is no more datetime-like, since it captures multiple time periods. That's why it's required here to pass the frequency freq to the Portfolio class in order to be able to compute performance metrics such as the Sharpe ratio.

The index hierarchy of the final performance series can be then used to group the performance by any feature, such as window pair, symbol, and time period.

In [ ]:
mean_return = pf.total_return().groupby(['split_idx', 'symbol']).mean()
mean_return.unstack(level=-1).vbt.barplot(
    xaxis_title='Split index', yaxis_title='Mean total return',
    legend_title_text='Symbol')

KeyError: ignored

There is much more to backtesting than simply stacking columns: vectorbt offers functions for most parts of a backtesting pipeline - from building indicators and generating signals, to modeling portfolio performance and visualizing results.

# **2. Primeiros passos com a VectorBT**

## 2.1. Baixar os dados de cotações

In [ ]:
fast_ma = vbt.MA.run(btc_price, 10, short_name='fast')
slow_ma = vbt.MA.run(btc_price, 30, short_name='slow')

entries = fast_ma.ma_crossed_above(slow_ma)
entries

Date
2021-01-01 00:00:00+00:00    False
2021-01-02 00:00:00+00:00    False
2021-01-03 00:00:00+00:00    False
2021-01-04 00:00:00+00:00    False
2021-01-05 00:00:00+00:00    False
                             ...  
2021-12-27 00:00:00+00:00    False
2021-12-28 00:00:00+00:00    False
2021-12-29 00:00:00+00:00    False
2021-12-30 00:00:00+00:00    False
2021-12-31 00:00:00+00:00    False
Freq: D, Length: 365, dtype: bool

In [ ]:
exits = fast_ma.ma_crossed_below(slow_ma)
exits

Date
2021-01-01 00:00:00+00:00    False
2021-01-02 00:00:00+00:00    False
2021-01-03 00:00:00+00:00    False
2021-01-04 00:00:00+00:00    False
2021-01-05 00:00:00+00:00    False
                             ...  
2021-12-27 00:00:00+00:00    False
2021-12-28 00:00:00+00:00    False
2021-12-29 00:00:00+00:00    False
2021-12-30 00:00:00+00:00    False
2021-12-31 00:00:00+00:00    False
Freq: D, Length: 365, dtype: bool

In [ ]:
pf = vbt.Portfolio.from_signals(btc_price, entries, exits)
pf.total_return()

0.06837363892891674

# **3. Algumas funções gerais da biblioteca *vectorbt***